# Function Decorators and Closures

Function decorators let us "mark" functions in the source code to enhance their behaviour in some way. This is powerful stuff, but mastering it requires understanding closures.

Aside from their application in decorators, closures are also essential for effective asynchronous programming with callbacks, and for coding in a funcitonal style wenever it makes sense.

## Decorators 101

A decorator is a callable that takes another function as an arguement. The decorator may perform  some processing with the decorated function, and returns it or replace it with anther function or callable object.

In [1]:
def deco(func):
    def inner():
        print('running inner')
    return inner

In [3]:
@deco
def target():
    print('running target')

In [4]:
target()

running inner


In [6]:
target

<function __main__.deco.<locals>.inner()>

1. deco returns its inner function obect
2. target i s decorated by deco
3. invoking the decorated target actually runs inner
4. inspection reveals that target is now a reference to inner


Strictly speaking,, decorators are just syntatic sugar.

To summarize: the first crucial fact about  decorators is that they have the power to replace the decorated function with a different one. The second crucial fact is that they are executed immediately when a module is loaded. This is explained next.

## When Python Executes Decorators

A key feature of decorators is that they run right after the decorated function is defined. That is usually at import time, (ie when a module is loaded by python)

In [11]:
registry = []

def register(func):
    print(f'running register {func}')
    registry.append(func)
    return func

@register
def f1():
    print('funning f1()')
    
@register
def f2():
    print('running f2()')
    

def f3():
    print('running f3()')
    
    
def main():
    print('running main')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    

main()

running register <function f1 at 0x00000159DFE78CA0>
running register <function f2 at 0x00000159DFE78820>
running main
registry -> [<function f1 at 0x00000159DFE78CA0>, <function f2 at 0x00000159DFE78820>]
funning f1()
running f2()
running f3()


1. registry will hold references to functions decorated by @register
2. register takes a function as an argument
3. Display what function is being decorated
4. include func in registry
5. Return funct: we must return a function; here we return the same recieved as argument
6. f1 and f2 are decorate by @register
7. f3 is not decorated
8. main displayes the registry, then calls f1(), f2(), f2()


Note that register runs(twice) before any other function in the module. When register is called, it recieves as an argument the function object being decorated.

After the module is loaded, the registry holds references to the two deccorated functions: f1 and f2. These functionsm as well as f3, are only executed ewhen explicity called by main. 

If registration.py is imported (and not run as a script), the output is this:

    ```py
    import resigstration
    running register <function f1 at 0x00000159DFE78CA0>
    running register <function f2 at 0x00000159DFE78820>
    ```

At this rime if you look at the registry, here is what you get:

    ```py
    registration.registry
    [<function f1 at 0x00000159DFE78CA0>, <function f2 at 0x00000159DFE78820>]
    ```

The main point is to emphasize that function decorators are executed as soon as the module is imported, but the decorated functions only run when they are explicitly invoked. This is highlights the difference between import time and runtime.

In real code decorators...
- A real decorator is usually defined in one module and applied to functions in other modules
- Most decorators define an inner function and return it

## Decorator-Enhanced Strategy PAttern

Recall that our main issue with Example 6-6 is the repetition of the function names in their definitions and then in the promos list used by the best_promo function to deter‐ mine the highest discount applicable. The repetition is problematic because someone may add a new promotional strategy function and forget to manually add it to the promos list—in which case, best_promo will silently ignore the new strategy, introducing a subtle bug in the system. Example 7-3 solves this problem with a registration decorator.

In [ ]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func


@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantit >= 20:
            discount += item.total() * 0.1
    return discount

@promotion 
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0 

def best_promo(order):
    """Select best discount available""" 
    return max(promo(order) for promo in promos)

1. promos list starts empty
2. pomotion decorator returns promo_func unchanged, after adding it to the promos list
3. any  function decorated by @promotion wil be added to promos
4. no changes needed to best_promos, because it relies on the promos list


Adcantages:
- The promotion strategy functions don't have to use special names (ie they don't have to use the _promos suffix)
- The @promotion decorator highlights the purpose of the decorated function, and also makes it easy to temporarily disable a promotion: just comment out the decorator
- Promotional discount strategies may be defined in other modules, anywhere in the system, as long as the @promotion decorator is applied to them


Most decorators do change the decorated function. They usually do it by defining an inner function and returning it to replace the decorated function. Code that uses inner functions almost always depends on closures to operate correctly.

## Varibale Scope Rules

In [13]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [14]:
b = 6
f1(3)

3
6


In [16]:
b = 6
def f2(a):
    print(a)
    print(b)
    b =  6 
    
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

When python compiles the function, it decides that b is a local variable because it is assigned within the function. 

Python does not require you to declare varibales but assumes that a variable assigned in the body of a function is local. 

In [17]:
b = 6
def f2(a):
    global b
    print(a)
    print(b)
    b =  6 
    
f2(3)

3
6


In [18]:
b

6

In [19]:
from dis import dis

dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


1. Load global name print
2. Load locak name a
3. Load global name b

In [20]:
dis(f2)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (6)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


1. Load local name b This shows that the compiler considers b a local variable, even if the assignment to be occurs later, because the nature of the variable - whether it is local or not- cannot change the body of the function.

## Closures 

A closure is a function with an extened scope that encompasses nonglobal variables referenced in the body of the function but not defined there. It does not matter whether the function is anonymous or not; what matters is that it can access nonglobal variables that are defined out of its body

In [22]:
class Averager:
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [23]:
avg = Averager()
avg(10)

10.0

In [24]:
avg(25)

17.5

Functional implementation, using the higher-order function make_averager

In [25]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

When invoked make_averager returns an averager funcction object. Each time and averager is called, it appends the passed argument to the series and computes the current average.

In [26]:
avg = make_averager()
avg(10)

10.0

In [27]:
avg(12)

11.0

Note the similarities of the example; as we call Averager() and maker_averager() to get a callable object avg that will update the historical series and calculate the current mean. 

It is obvious that the Averager class stores the history in self.series instance attribute. But where does the avg_function store the series?

Note that series is a local variable of make_averager because the initialization series = [] happens in the body of that function. But when avg(10) is callled, make_averager has already returned, and its local scope is long gone.

Within averager, series is a free variable. This is a technical term meaning a variable that is not bound in the local scope.

Inspecting the returned averager objects shows how Python keeps the names of local and free variables in the `__code__` attribute that represents the compiled body of the function.

In [28]:
avg.__code__.co_varnames

('new_value', 'total')

In [29]:
avg.__code__.co_freevars

('series',)

The binding for series is kept in the `__closure__` attribute of the returned function avg. Each item in `avg.__closure__` corresponds to a name in `avg.__code__.co__freevars`. These items are cells, and they have an attribute called `cell_contents` where the actual value can be found.

In [30]:
avg.__code__.co_freevars

('series',)

In [31]:
avg.__closure__

(<cell at 0x00000159E0130C10: list object at 0x00000159DFDAE840>,)

In [33]:
avg.__closure__[0].cell_contents

[10, 12]

To summarize: a closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer avilable.

## The nonlocal Declaration

Out previous implementation of make_averager was not efficient. We stored all the values in the historical series and computed their sum ever time averager was called. A better implementation would just store the total and the number of items so fat, and compute the mean from these two numbers.

In [34]:
def make_averager():
    count = 0 
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [35]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

The problem is that the statement, count += 1 atually means the same as count = count + 1, when count is a number or any immutable type. So we are actually assigning to count in the body of averager, and that makes it a local variable. The same problem affects the total variable. We did not have this problem before because we never assigned to the series name, we only appened to it, taking advantage of the fact that lists are mutable

But with immutable types like numbers, strings, tuples, etc., all you can do is read, but never update. If you try to rebind them, as in count = count + 1, then you are implicitly creating a local variable count. It is no longer a free variable, and therefore it is not saved in the closure.

To work around this, the `nonlocal` declaration was introduced in Python 3. It lets you flag a variable as a free variable even when it is assigned a new value within the function. If a new value is assigned to a nonlocal variable, the binding stored in the closure is changed. 

In [37]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

In [38]:
avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


## Implementing a Simple Decorator

Make a decorator that clocks every invocation of the decorated function and prints the elapsed time, the arguments passed, and the results of the call.

In [39]:
import time
from typing import Callable

def clock(func: Callable):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args) # 2
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'{elapsed}, {name}, {arg_str}, {result}')
        return result
    
    return clocked

1. Define inner function clocked to accept any number of positional arguemments
2. This only works because the closure for clocked encompasses the func free variable
3. Return the inner function to replace the decorated function

In [40]:
import time


@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
0.1347596000014164, snooze, 0.123, None
**************************************** Calling factorial(6)
1.5999994502635673e-06, factorial, 1, 1
0.0008985999993456062, factorial, 2, 2
0.0009608000000298489, factorial, 3, 6
0.0010029999994003447, factorial, 4, 24
0.0010421999995742226, factorial, 5, 120
0.0010817000002134591, factorial, 6, 720
6! = 720


### How it Works

clock gets the factorial function as its func argument. It then creates and returns the clocked function, which the python interpreter assigns to factorialbehind the scenes.

In [41]:
factorial.__name__

'clocked'

So factorial now actually holds a reference to the clocked function. From now on, each time factorial(n) is called, clocked(n) gets executed. In essence, clocked doesthe following: 

1. Records the initial time t0
2. Calls the original factorial, saving the result
3. Computes the elapsed time
4. Formats and prints the collected data
5. Returns the result save to step 2


This is the typical behavior of a decorator; it replaces the decorated function with a new function that accepts the same arguements and (usually) returns whatever the decorated function was supposed to return, while also doing some extra processing.

The clock decorator implemented above has a few shortcomings: it does not support keyword arguments, and it masks the `__name__` and `__doc__` of the decorated function.

Below; use the `functool.wraps` decorator to copy the relevant attributes from func to clocked. Also keyword arguments are correctly handled

If using a decorator always meant losing this information about a function, it would be a serious problem. That's why we have functools.wraps. This takes a function used in a decorator and adds the functionality of copying over the function name, docstring, arguments list, etc. And since wraps is itself a decorator, the following code does the correct thing

In [1]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_list = []
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_list.append(', '.join(pairs))
        arg_str = ', '.join(arg_list)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    return clocked

## Decorators in the Standard Library

Python has three built-in functions that are designed to decorate methods: property, classmethod, and staticmethod.

Another frequently seen decorator is functool.wraps, a helper for building well-behaved decorators.

Two of the most interesting decorators in the standard library are `lru_cache` and `singledispatch`.

### Memoization with functools.lru_cache

A very practical decorator is `functools.lru_cache`. It implements memoization: and optimization technique that works by saving the results of previous invocations of an expensive function, avoiding repeat computations on previously used arguements. The letters LRU stand for `Least Recenlt Used`, meaning that the growth of the cache is limited by discarding the entries that have not been read for a while.

A good demonstration is to apply lru_cache to the painfully slow recursive function to generate the nth fibonacci sequence.

In [2]:
@clock 
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [6]:
print(fibonacci(4))

[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00100660s] fibonacci(2) -> 1 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(2) -> 1 
[0.00000000s] fibonacci(3) -> 2 
[0.00100660s] fibonacci(4) -> 3 
3


The waste is obvious, fibnoacci is called on the same n multiple times. Now with lru_cache

In [17]:
import functools

@functools.lru_cache()
@clock 
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

1. `lru_cache` must be invoked as a regular function, with the parentheses, becuase it accepts configuration parameters
2. This is an example of stacked decorators @lru_cache() is applied on the function returned by @clock

In [18]:
print(fibonacci(4))

[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00100279s] fibonacci(2) -> 1 
[0.00000000s] fibonacci(3) -> 2 
[0.00100279s] fibonacci(4) -> 3 
3


Execution time is halved, and the function is called only once for each value of n.

It’s important to note that lru_cache can be tuned by passing two optional arguments. It's full signature is:

       functools.lru_cache(maxsize=120, typed=False)

The maxsize argument determines how many call results are stored. After the cacge is full, older  results are discarded to make room. For optimal performance, maxsize should be a power of 2. The typed argument, if set to True, stores results of different argument types seperately, ie distinguishing between float and integer arguments that are normall considered equal like 1 and 1.0. . By the way, because lru_cache uses a dict to store the results, and the keys are made from the positional and keyword arguments used in the calls, all the arguments taken by the decorated function must be hashable.

### Generic Functions with Single Dispatch

Imagine we are creating a tool to debug web applications. We want to be able to generate HTML displays for different types of Python objects

In [3]:
import html


def htmlize(object):
    content = html.escape(repr(object))
    return f"<pre>{content}</pre>"

That will work for any Python type, but now we want to extend it to generate custom displays for some types:
- str: replace embedded newline characters with `<br>\n` and use `<p>` tags instead of `<pre>`.
- int: show the number in decimal and hexadecimal.
- list: output an HTML list, formatting each item according to its type

The new `functools.singledispatch` decorator in Python 3.4 allows each module to contribute to the overall solution, and lets you easily provide a specialized function even for classes that you can’t edit. If you decorate a plain function with `@singledispatch`, it becomes a generic function: a group of functions to perform the same operation in different ways, depending on the type of the first argument

In [10]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return f"<pre>{content}</pre>"


@htmlize.register(str)
def _(text):
    content = html.escape(text).replace("\n", "<br>\n")
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)


@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

1. @singledispatch marks the base function that handles the object type
2. each specialized function is decorated with @base_function.register(type) 
3. the name of the specialized functions is irrelevant; _ is a good choice to make this clear 
4. for each additional type to receive special treatment, register a new function 
5. you can stack several register decorators to support different types with the same function

When possible, register the specialized functions to handle ABCs (abstract classes) suchas numbers.Integral and abc.MutableSequence instead of concrete implementations like int and list. This allows your code to support a greater variety of compatible types.

## Stacked Decorators

When two decorators @d1 and @d2 are applied to a function f in that order, the result is the same as f = d1(d2(f)) 

## Parameterized Decorators

When parsing a decorator in source code, Python takes the decorated function and passes it as the first argument to the decorator function. 

To make a decorator that accepts other arguments, make a decorator factory that takes those arguments and returns a decorator, which is then applied to the function to be decorated.  

In [6]:
registry = []

def register(func):
    print(f'running register {func}')
    registry.append(func)
    return func 

@register
def f1(): 
    print('running f1')

print('running main()')
print('registry ->', registry)
f1()

running register <function f1 at 0x00000285FFFDBA60>
running main()
registry -> [<function f1 at 0x00000285FFFDBA60>]
running f1


### A Parameterized Registration Decorator 

In order to make it easy to enable or disable the function registration performed by register, we’ll make it accept an optional active parameter which, if False, skips registering the decorated function

Conceptually, the new register function is not a decorator but a decorator factory. When called, it returns the actual decorator that will be applied to the target function

In [1]:
registry = set()
def register(active=True): 
    def decorate(func): 
        print(f'running register({active})->decorate({func})')
        if active: 
            registry.add(func)
        else:
            registry.discard(func) 
        return func 
    return decorate 
    
@register(active=False) 
def f1():
    print('running f1()')
    
@register() 
def f2(): 
    print('running f2()')

def f3():
     print('running f3()')

running register(False)->decorate(<function f1 at 0x000001703944C820>)
running register(True)->decorate(<function f2 at 0x000001703944CA60>)


In [2]:
print(registry)

{<function f2 at 0x000001703944CA60>}


1. registry is now a set, so adding and removing functions is faster
2. register takes an optional keyword
3. the decorate inner function is the actual decorator
4. register func only if the active argument is True
5. if not active and func in registry remove it
6. because decorate is a decorator, it must return a function
7. register is our decorator factory, so it returns decorate
8. the @register factory must be invoked as a function, with the desired parameters
9. if no parameters are passed, register must still be called as a function, @register() to return the actual decorator decorate

If instead of using the @ syntax, we used register as a regular function to decorate a function, the syntax would be `register()(func)`

In [3]:
register()(f3)

running register(True)->decorate(<function f3 at 0x000001703944C280>)


<function __main__.f3()>

### Parameterized clock decorator

In [4]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'
def clock(fmt=DEFAULT_FMT): 
    def decorate(func): 
        def clocked(*_args): 
            t0 = time.time()
            _result = func(*_args) 
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args) 
            result = repr(_result) 
            print(fmt.format(**locals())) 
            return _result 
        return clocked 
    return decorate

In [5]:
@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(0.123)

[0.13798523s] snooze(0.123) -> None
[0.13699865s] snooze(0.123) -> None
[0.13799834s] snooze(0.123) -> None


1. clock is our parameterized decorator factory
2. decorate is the actual decorator
3. clocked wraps the decorated function
4. _result is the actual result of the decorated function
5. _args holds the actual arguments of clocked, while args is str used for display
6. result is the str representation of _result, for display
7. using **locals() here allows any local variable of clocked to be referenced in the fmt
8. clocked will replace the decorated function, so it should return whatever that function returns
9. decorate returns clocked
10. clock returns decorate
11. in this self test, clock() is called without arguments, so the decorator applied will use the default format str

# Chapter Summary
 
Learned about `variable scope rules` such as `global` and `local`.

Learned about closures, a function with an extended scope that encompasses `nonglobal` variables referenced in the body of the function but not defined there

Parameterized decorators almost always involve at least two nested functions, maybe more if you want to use `@functools.wraps` to produce a decorator that provides better support for more advanced techniques. One such technique is `stacked decorators,` which we briefly covered.

We also visited two awesome function decorators provided in the functools module of standard library: `@lru_cache() and @singledispatch`